In [4]:
! pip install -q langgraph-checkpoint-sqlite

In [1]:
import sqlite3
from typing import TypedDict

from langchain.tools import tool
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt


In [2]:
class AgentState(TypedDict):
    messages: list[dict]

In [3]:
import sys
sys.path.append('..')

from llm_wrapper import get_chat_llm

llm = get_chat_llm()

✓ Opik tracer initialized for project: customer-support


In [ ]:
doc agent
db agent
hitl

user_query -> agent.invoke(user_query, thread id) 
hitl -> interrupt - stop the graph execution and send the details to human
state -> dumped to storage
feedback requested to human agent


human agent -> resume with feedback
call the agent again with same thread id
graph will continue from last state with feedback



In [4]:
@tool
def send_email(to: str, subject: str, body: str):
    """Send an email to a recipient."""

    # Pause before sending; payload surfaces in result["__interrupt__"]
    response = interrupt({
        "action": "send_email",
        "to": to,
        "subject": subject,
        "body": body,
        "message": "Approve sending this email?",
    })

    if response.get("action") == "approve":
        final_to = response.get("to", to)
        final_subject = response.get("subject", subject)
        final_body = response.get("body", body)

        # Actually send the email (your implementation here)
        print(f"[send_email] to={final_to} subject={final_subject} body={final_body}")
        return f"Email sent to {final_to}"

    return "Email cancelled by user"


In [7]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(llm, tools=[send_email], checkpointer=InMemorySaver())

In [15]:
config = {"configurable": {"thread_id": "email-workflow-1"}}
initial = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "lets Send an email to alice@vidvatta.in about the meeting tomorrow at 10 AM."}
        ]
    },
    config=config,
)

In [16]:
initial

{'messages': [HumanMessage(content='lets Send an email to alice@vidvatta.in about the meeting tomorrow at 10 AM.', additional_kwargs={}, response_metadata={}, id='f2c4d6a1-e692-4008-bcdd-fc879b50bbbe'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 72, 'total_tokens': 115, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_3dcd5944f5', 'id': 'chatcmpl-Cjfg1CxLb2vruCwTK8cXHZa3IqogC', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': '

In [17]:
# Resume with approval and optionally edited arguments
resumed = agent.invoke(
    Command(resume={"action": "disapprove", "subject": "Updated subject"}),
    config=config,
)
print(resumed["messages"][-1]) 

content='It seems the email was not sent. Would you like me to try sending it again?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 126, 'total_tokens': 146, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_3dcd5944f5', 'id': 'chatcmpl-CjfgD0uuJsC6oHEYVVvuViX3TQo95', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate':